In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)


# Dataset

In [2]:
df = pd.read_csv("../data/code-review-dataset-clear.csv")

In [3]:
X_train, X_val = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df["is_toxic"]
)

train_ds = Dataset.from_pandas(
    X_train[["message", "is_toxic"]].rename(columns={"message": "text", "is_toxic": "labels"}),
    preserve_index=False
)
val_ds = Dataset.from_pandas(
    X_val[["message", "is_toxic"]].rename(columns={"message": "text", "is_toxic": "labels"}),
    preserve_index=False
)

# RoBERTa


In [4]:
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
    )

train_ds = train_ds.map(tokenize_batch, batched=True)
val_ds   = val_ds.map(tokenize_batch, batched=True)

cols_to_keep = ["input_ids", "attention_mask", "labels"]
train_ds = train_ds.remove_columns([c for c in train_ds.column_names if c not in cols_to_keep])
val_ds   = val_ds.remove_columns([c for c in val_ds.column_names   if c not in cols_to_keep])


Map: 100%|██████████| 2581/2581 [00:00<00:00, 15135.10 examples/s]


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "non_toxic", 1: "toxic"},
    label2id={"non_toxic": 0, "toxic": 1},
)


In [6]:

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", pos_label=1, zero_division=0)
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f1}



In [ ]:
args = TrainingArguments(
    output_dir="./codeberta_toxic_cls",
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,
    logging_steps=50,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [8]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.212400,0.223109,0.906238,0.810427,0.678571,0.738661
2,0.188700,0.250727,0.917861,0.841121,0.714286,0.772532
3,0.144900,0.302782,0.913987,0.772201,0.793651,0.782779
4,0.131200,0.352796,0.917086,0.798354,0.769841,0.783838
5,0.140900,0.376115,0.916312,0.788000,0.781746,0.784861


TrainOutput(global_step=3230, training_loss=0.18466243374827476, metrics={'train_runtime': 151.9981, 'train_samples_per_second': 339.544, 'train_steps_per_second': 21.25, 'total_flos': 3394790391782400.0, 'train_loss': 0.18466243374827476, 'epoch': 5.0})

In [9]:
eval_metrics = trainer.evaluate()
print(eval_metrics)

{'eval_loss': 0.3761151134967804, 'eval_accuracy': 0.9163115071677644, 'eval_precision': 0.788, 'eval_recall': 0.7817460317460317, 'eval_f1': 0.7848605577689243, 'eval_runtime': 1.4412, 'eval_samples_per_second': 1790.922, 'eval_steps_per_second': 56.205, 'epoch': 5.0}
